In [1]:
# GNU LESSER GENERAL PUBLIC LICENSE
# Version 3, 29 June 2007.
# Copyright (C) 2007 Free Software Foundation, Inc. <http://fsf.org/>.
# Everyone is permitted to copy and distribute verbatim copies of this 
# license document, but changing it is not allowed.

# Creating Voronoi Tesselations
## Automated Simulation in GIS
---

### James Gaboardi, 2016

In [2]:
import numpy as np
import pysal as ps
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import multiprocessing as mp
import shapely
import time
from scipy.spatial import Voronoi, voronoi_plot_2d
%matplotlib inline

mpl.rcParams['figure.figsize'] = 20,20  #set the default map size
mpl.rcParams['patch.linewidth'] = 0.5  #set default polygon line width

np.random.seed(352)

# Local path on user's machine
path = '/Users/jgaboardi/Algorithms_etc./Data/'

/Users/jgaboardi/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [ ]:
def sq(x):
    for i in range(1000):
        return x*x



In [ ]:

def sq(x):
    for i in range(1000):
        return x*x


cores = mp.cpu_count()
print cores,' Cores in each Node '
p = mp.Pool(processes=cores)
count = range(20)
print len(count), ' total # of integers '
np.random.seed(352)

map_time = []
multi_time = []



t1_multi = time.time()            # multi
output_mult = p.map(sq, Ai)
t2_multi = time.time() - t1_multi




Map_v_Multi = pd.DataFrame(index=count)
Map_v_Multi['Map'] = map_time
Map_v_Multi['Multi'] = multi_time
Map_v_Multi.index.name = 'Count'
Map_v_Multi.to_csv('MP_Sing.csv')

In [3]:
t1_shp_reads = time.time()

# Waverly Hills streets and bounding box
streets = gpd.read_file(path+'Waverly_Trim/Waverly.shp')
streets.to_crs(epsg=2779, inplace=True) # NAD83(HARN) / Florida North
streets.to_file(path+'waverly/waverly.shp')
shp_waverly = ps.open(path+'waverly/waverly.shp')
b_box = shp_waverly.bbox

'''
# Leon County streets and bounding box
streets = gpd.read_file(path+'Leon_County/LCSTSEG.shp')
streets.to_crs(epsg=2779, inplace=True) # NAD83(HARN) / Florida North
streets.to_file(path+'leon/leon.shp')
shp_Leon = ps.open(path+'leon/leon.shp')
b_box = shp_Leon.bbox
'''
t2_shp_reads = time.time()-t1_shp_reads
print t2_shp_reads

0.396848917007


In [4]:
# Individual street buffers in meters
intial_buffer = streets.buffer(50)  
intial_buffer[:5]

0    POLYGON ((623124.0407875385 164212.5102460865,...
1    POLYGON ((622546.5762248621 163255.235891574, ...
2    POLYGON ((623559.7869314271 165528.3883790929,...
3    POLYGON ((623200.6855916528 164368.5069398382,...
4    POLYGON ((623558.3641320666 165459.8080567809,...
dtype: object

In [5]:
# Union of individual buffers
union_buffer = intial_buffer.unary_union  
union_buffer = gpd.GeoSeries(union_buffer)
union_buffer.crs = streets.crs
union_buffer = gpd.GeoDataFrame(union_buffer, crs=streets.crs)
union_buffer.columns = ['geometry']

In [6]:
# Create convex hull buffer union
convex_hull_of_union = gpd.GeoDataFrame(union_buffer.convex_hull)
convex_hull_of_union.columns = ['geometry']

In [ ]:
# Generate (n) simulated households
n = 100000
t1_allpoints = time.time()
np.random.seed(352)
x = np.random.uniform(b_box[0], b_box[2], n)
np.random.seed(850)
y = np.random.uniform(b_box[1], b_box[3], n)  
simulated_coords = zip(x,y)
simulated_coord_points = [shapely.geometry.Point(xy_coord) for xy_coord in simulated_coords]
simulated_households = gpd.GeoDataFrame(simulated_coord_points, crs=streets.crs)
simulated_households.columns = ['geometry']
t2_allpoints = time.time()-t1_allpoints
print t2_allpoints, 'seconds to simulate households in Waverly'

1.4156639576 seconds to simulate households in Waverly


In [ ]:
# No Multi Processing

# Identify households within the buffer
t1_intersection_no_multi = time.time()
intersection = [union_buffer['geometry'].intersection(hh) for hh in simulated_households['geometry']]
t2_intersection_no_multi = time.time()-t1_intersection_no_multi
print t2_intersection_no_multi, 'seconds to identify households within the 50m street buffer No MultiProcessing'
intersection = gpd.GeoDataFrame(intersection, crs=streets.crs)
intersection.columns = ['geometry']

# Ignore households not in the buffer --> shapely.geometry.collection.GeometryCollection
households_gdf = intersection[intersection.geom_type != 'GeometryCollection']
households_gdf[:5]

In [ ]:
# Multi Processing


t1_intersection_multi = time.time()

intersection = [union_buffer['geometry'].intersection(hh) for hh in simulated_households['geometry']]



t2_intersection_multi = time.time()-t1_intersection_multi


print t2_intersection_multi, 'seconds to identify households within the 50m street buffer with MULTI Processing'

------------